In [1]:
import cv2
import math
import numpy as np

class EuclideanDistTracker:
    def __init__(self):
        # Store the center positions of the cars
        self.center_points = {}
        
        # each time a new car id detected, counter will increase by one
        self.id_count = 0

    def update(self, cars_rect):
        # Cars' bounding boxes and ids
        cars_bbs_ids = []

        # Get center point of newly detected car
        for rect in cars_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Check if car was detected already
            car_already_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 80:
                    self.center_points[id] = (cx, cy)
                    print(self.center_points)
                    cars_bbs_ids.append([x, y, w, h, id])
                    car_already_detected = True
                    break

            # If new car, assign ID to that car
            if car_already_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                cars_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for car in cars_bbs_ids:
            _, _, _, _, car_id = car
            center = self.center_points[car_id]
            new_center_points[car_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return cars_bbs_ids

class CarTracker():
    def __init__(self):
        self.awayCityCtr = []
        self.toCityCtr = []
        self.center_points = {}
        self.dirVal = {}
        

    def check_x_coords(self, cars_bbs_ids):
        for car in cars_bbs_ids:
            x, y, _, _, car_id = car

            if car_id in self.center_points.keys():
                if x < self.center_points[car_id] and x < 550 and x > 500 and y > 350 and y < 450:
                    self.dirVal[car_id] += x-self.center_points[car_id] 
                    self.center_points.update({car_id: x})

                elif x > self.center_points[car_id] and x < 550 and x > 500 and y > 350 and y < 450:
                    self.dirVal[car_id] += x-self.center_points[car_id] 
                    self.center_points.update({car_id: x})
                    
                else:
                    continue

            else:
                self.center_points.update({car_id: x})
                self.dirVal.update({car_id: 0})
                
            if self.dirVal[car_id] > 0:
                if car_id in self.toCityCtr:
                    self.toCityCtr.remove(car_id)
                elif car_id in self.awayCityCtr:
                    continue
                else:
                    self.awayCityCtr.append(car_id)
                
            elif self.dirVal[car_id] < 0:
                if car_id in self.awayCityCtr:
                    self.awayCityCtr.remove(car_id)
                elif car_id in self.toCityCtr:
                    continue
                else:
                    self.toCityCtr.append(car_id)
    
# Create tracker object
tracker1 = EuclideanDistTracker()
checker1 = CarTracker()

# create background subtractor object
backSub = cv2.createBackgroundSubtractorMOG2()

# kernel for image dilation
kernel = np.ones((4,4),np.uint8)

# Open the video file
video= cv2.VideoCapture('Traffic_Laramie_1.mp4')

# check if the video opened successfully
if (video.isOpened()== False): 
    print("Error opening video file")

# Read the first frame
ret, frame1 = video.read()

# get width and height of video frame
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# define dimensions for region of interest(ROI) rectangle
roiX = 0
roiY = int(height/2-40)
roiW = width
roiH = int(height/2+40)

# initialize car position storage
first_loc = []
next_loc = []

# Convert first frame to grayscale
prev_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# Loop through the video
while True:
    # Read the next frame
    ret, frame2 = video.read()
    
    # If there is no next frame, break out of the loop
    if not ret:
        break
    
    detections = []
    
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    # Perform frame differencing
    frame_diff = cv2.absdiff(prev_gray, gray)
    
    # apply background subtraction to ROI
    fgMask = backSub.apply(frame_diff)
    
    # remove noise
    image = cv2.GaussianBlur(fgMask, (25,25), 0)

    # apply thresholding to create binary image
    thresh = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)[1]
    
    # image dilation
    dilated = cv2.dilate(thresh,kernel,iterations = 1)
    
    # find contours in binary image
    contours, hierarchy = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # loop through contours
    for cnt in contours:
        # calculate contour area
        ctArea = cv2.contourArea(cnt)

        # draw bounding box around contour
        (x, y, w, h) = cv2.boundingRect(cnt)
        if y > height/2-40 and ctArea > 3500:
            cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 255, 0), 2)
            detections.append([x, y, w, h])

            
    cars_ids = tracker1.update(detections)
    checker1.check_x_coords(cars_ids)
    
    for car_id in cars_ids:
        x, y, w, h, id = car_id
        cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # draw ROI rectangle on frame
    cv2.rectangle(frame2, (roiX, roiY), (roiX + roiW, roiY + roiH), (0, 0, 255), 2)
    cv2.rectangle(frame2, (500, 350), (550, 450), (0, 255, 0), 2)
    
    # Display the resulting frame
    cv2.imshow('Frame Difference', dilated)
    cv2.imshow('Frame', frame2)
    
    # Wait for user input to exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the capture and destroy any OpenCV windows
video.release()
cv2.destroyAllWindows()

{0: (1004, 538)}
{0: (1000, 535)}
{0: (998, 536)}
{1: (991, 530)}
{1: (988, 528)}
{1: (985, 526)}
{1: (985, 517)}
{1: (980, 522)}
{1: (977, 518)}
{1: (974, 516)}
{1: (971, 515)}
{1: (970, 508)}
{1: (966, 506)}
{1: (962, 505)}
{1: (959, 502)}
{1: (957, 500)}
{1: (956, 498)}
{1: (944, 496)}
{1: (937, 494)}
{1: (931, 494)}
{1: (927, 497)}
{1: (923, 490)}
{1: (914, 491)}
{1: (909, 489)}
{1: (906, 488)}
{1: (901, 478)}
{1: (899, 477)}
{1: (884, 475)}
{1: (879, 473)}
{1: (872, 472)}
{1: (869, 476)}
{1: (866, 469)}
{1: (857, 470)}
{1: (849, 469)}
{1: (843, 468)}
{1: (837, 461)}
{1: (834, 460)}
{1: (821, 461)}
{1: (814, 461)}
{1: (806, 459)}
{1: (800, 458)}
{1: (798, 457)}
{1: (787, 454)}
{1: (778, 453)}
{1: (771, 451)}
{1: (766, 449)}
{1: (763, 449)}
{1: (747, 445)}
{1: (742, 444)}
{1: (734, 443)}
{1: (728, 441)}
{1: (724, 441)}
{1: (712, 437)}
{1: (704, 436)}
{1: (698, 435)}
{1: (690, 434)}
{1: (687, 433)}
{1: (678, 431)}
{1: (669, 429)}
{1: (661, 429)}
{1: (656, 427)}
{1: (657, 425)}
{1: (6

{2: (248, 491), 3: (580, 432), 4: (39, 365)}
{2: (248, 491), 3: (563, 429), 4: (39, 365)}
{2: (248, 491), 3: (563, 429), 4: (36, 368)}
{2: (253, 491), 3: (563, 429), 4: (36, 368)}
{2: (253, 491), 3: (553, 428), 4: (36, 368)}
{2: (253, 491), 3: (553, 428), 4: (35, 371)}
{2: (258, 492), 3: (553, 428), 4: (35, 371)}
{2: (258, 492), 3: (544, 427), 4: (35, 371)}
{2: (258, 492), 3: (544, 427), 4: (35, 374)}
{2: (263, 493), 3: (544, 427), 4: (35, 374)}
{2: (263, 493), 3: (534, 426), 4: (35, 374)}
{2: (263, 493), 3: (534, 426), 4: (33, 376)}
{2: (263, 493), 3: (534, 426), 4: (33, 376)}
{2: (263, 493), 3: (532, 426), 4: (33, 376)}
{2: (263, 493), 3: (532, 426), 4: (33, 376)}
{2: (271, 494), 3: (532, 426), 4: (33, 376)}
{2: (271, 494), 3: (515, 425), 4: (33, 376)}
{2: (271, 494), 3: (515, 425), 4: (31, 377)}
{2: (274, 495), 3: (515, 425), 4: (31, 377)}
{2: (274, 495), 3: (506, 424), 4: (31, 377)}
{2: (274, 495), 3: (506, 424), 4: (30, 378)}
{2: (281, 496), 3: (506, 424), 4: (30, 378)}
{2: (281, 

{2: (448, 515), 4: (135, 423)}
{2: (448, 515), 4: (135, 423)}
{2: (450, 515), 4: (135, 423)}
{2: (450, 515), 4: (136, 422)}
{2: (452, 515), 4: (136, 422)}
{2: (452, 515), 4: (136, 422)}
{2: (453, 515), 4: (136, 422)}
{2: (453, 515), 4: (136, 422)}
{2: (454, 515), 4: (136, 422)}
{2: (454, 515), 4: (136, 422)}
{2: (455, 515), 4: (136, 422)}
{2: (455, 515), 4: (136, 422)}
{2: (456, 515), 4: (136, 422)}
{2: (456, 515), 4: (136, 422)}
{2: (458, 515), 4: (136, 422)}
{2: (458, 515), 4: (136, 422)}
{2: (461, 515), 4: (136, 422)}
{2: (461, 515), 4: (136, 423)}
{2: (462, 515), 4: (136, 423)}
{2: (462, 515), 4: (136, 423)}
{2: (464, 515), 4: (136, 423)}
{2: (464, 515), 4: (136, 423)}
{2: (467, 515), 4: (136, 423)}
{2: (467, 515), 4: (136, 424)}
{2: (469, 515), 4: (136, 424)}
{2: (469, 515), 4: (136, 424)}
{2: (471, 515), 4: (136, 424)}
{2: (471, 515), 4: (136, 425)}
{2: (472, 515), 4: (136, 425)}
{2: (472, 515), 4: (136, 425)}
{2: (473, 515), 4: (136, 425)}
{2: (473, 515), 4: (136, 425)}
{2: (474

{7: (605, 522), 11: (43, 424)}
{7: (605, 522), 11: (43, 424)}
{7: (606, 522), 11: (43, 424)}
{7: (606, 522), 11: (44, 424)}
{7: (607, 522), 11: (44, 424)}
{7: (607, 522), 11: (45, 425)}
{7: (607, 522), 11: (45, 425)}
{7: (607, 522), 11: (45, 422)}
{7: (610, 522), 11: (45, 422)}
{7: (610, 522), 11: (46, 426)}
{7: (610, 522), 11: (46, 426)}
{7: (610, 522), 11: (46, 423)}
{7: (613, 522), 11: (46, 423)}
{7: (613, 522), 11: (46, 423)}
{7: (615, 522), 11: (46, 423)}
{7: (615, 522), 11: (47, 423)}
{7: (617, 522), 11: (47, 423)}
{7: (617, 522), 11: (48, 426)}
{7: (617, 522), 11: (48, 426)}
{7: (617, 522), 11: (48, 424)}
{7: (618, 522), 11: (48, 424)}
{7: (618, 522), 11: (48, 426)}
{7: (621, 522), 11: (48, 426)}
{7: (621, 522), 11: (49, 425)}
{7: (622, 522), 11: (49, 425)}
{7: (622, 522), 11: (49, 427)}
{7: (625, 522), 11: (49, 427)}
{7: (625, 522), 11: (50, 427)}
{7: (628, 522), 11: (50, 427)}
{7: (628, 522), 11: (50, 428)}
{7: (628, 522), 11: (50, 428)}
{7: (628, 522), 11: (52, 431)}
{7: (632

{20: (689, 469), 7: (923, 362)}
{20: (689, 469), 7: (921, 357)}
{20: (694, 469), 7: (921, 357)}
{20: (694, 469), 7: (920, 355)}
{20: (699, 478), 7: (920, 355)}
{20: (699, 478), 7: (919, 352)}
{20: (705, 477), 7: (919, 352)}
{20: (705, 477), 7: (918, 349)}
{20: (706, 477), 7: (918, 349)}
{20: (706, 477), 7: (917, 348)}
{20: (714, 474), 7: (917, 348)}
{20: (714, 474), 7: (916, 345)}
{20: (711, 485), 7: (916, 345)}
{20: (725, 485)}
{20: (729, 470)}
{20: (728, 482)}
{20: (736, 477)}
{20: (736, 476)}
{20: (741, 476)}
{20: (748, 474)}
{20: (748, 473)}
{20: (756, 470)}
{20: (759, 469)}
{20: (764, 468)}
{20: (769, 462)}
{20: (769, 461)}
{20: (776, 457), 21: (196, 499)}
{21: (198, 499), 20: (776, 457)}
{21: (198, 499), 20: (780, 455)}
{21: (200, 500), 20: (780, 455)}
{21: (200, 500), 20: (784, 452)}
{21: (212, 501), 20: (784, 452)}
{21: (212, 501), 20: (788, 451)}
{21: (210, 501), 20: (788, 451)}
{21: (210, 501), 20: (788, 456)}
{21: (217, 504), 20: (788, 456)}
{21: (217, 504), 20: (794, 445)}


...

{40: (178, 457), 41: (829, 311)}
{40: (178, 457), 41: (830, 312)}
{40: (182, 457), 41: (830, 312)}
{40: (182, 457), 41: (831, 314)}
{40: (184, 457), 41: (831, 314)}
{40: (184, 457), 41: (833, 316)}
{40: (185, 457), 41: (833, 316)}
{40: (185, 457), 41: (832, 316)}
{40: (187, 458), 41: (832, 316)}
{40: (187, 458), 41: (835, 319)}
{40: (187, 457), 41: (835, 319)}
{40: (187, 457), 41: (836, 321)}
{40: (188, 458), 41: (836, 321)}
{40: (188, 458), 41: (837, 324)}
{40: (188, 458), 41: (837, 324)}
{40: (188, 458), 41: (840, 326)}
{40: (189, 458), 41: (840, 326)}
{40: (189, 458), 41: (840, 326)}
{40: (190, 459), 41: (840, 326)}
{40: (190, 459), 41: (843, 331)}
{40: (191, 459), 41: (843, 331)}
{40: (191, 459), 41: (845, 334)}
{40: (193, 459), 41: (845, 334)}
{40: (193, 459), 41: (846, 337)}
{40: (193, 460), 41: (846, 337)}
{40: (193, 460), 41: (849, 339)}
{40: (193, 460), 41: (849, 340)}
{41: (853, 343)}
{41: (855, 346)}
{41: (857, 349)}
{41: (859, 353)}
{41: (860, 354)}
{41: (865, 359)}
{41: (8

{43: (662, 425), 42: (317, 400)}
{43: (662, 425), 42: (312, 399)}
{43: (656, 424), 42: (312, 399)}
{43: (656, 424), 42: (308, 399)}
{43: (648, 423), 42: (308, 399)}
{43: (648, 423), 42: (302, 399)}
{43: (648, 423), 42: (302, 399)}
{43: (648, 423), 42: (301, 399)}
{43: (638, 422), 42: (301, 399)}
{43: (638, 422), 42: (294, 399)}
{43: (632, 422), 42: (294, 399)}
{43: (632, 422), 42: (290, 398)}
{43: (626, 422), 42: (290, 398)}
{43: (626, 422), 42: (285, 398)}
{43: (618, 421), 42: (285, 398), 44: (1015, 454)}
{43: (618, 421), 42: (279, 398), 44: (1015, 454)}
{44: (1015, 455), 43: (618, 421), 42: (279, 398)}
{44: (1015, 455), 43: (619, 421), 42: (279, 398)}
{44: (1015, 455), 43: (619, 421), 42: (279, 398)}
{44: (1013, 453), 43: (619, 421), 42: (279, 398)}
{44: (1013, 453), 43: (608, 420), 42: (279, 398)}
{44: (1013, 453), 43: (608, 420), 42: (271, 397)}
{44: (1011, 451), 43: (608, 420), 42: (271, 397)}
{44: (1011, 451), 43: (601, 419), 42: (271, 397)}
{44: (1011, 451), 43: (601, 419), 42: 

{43: (325, 404), 42: (99, 382), 44: (900, 306)}
{43: (325, 404), 42: (97, 382), 44: (900, 306)}
{43: (325, 404), 42: (97, 382), 44: (896, 303)}
{43: (320, 403), 42: (97, 382), 44: (896, 303)}
{43: (320, 403), 42: (95, 381), 44: (896, 303)}
{43: (320, 403), 42: (95, 381), 44: (894, 301)}
{43: (319, 403), 42: (95, 381), 44: (894, 301)}
{43: (319, 403), 42: (94, 382), 44: (894, 301)}
{43: (319, 403), 42: (94, 382), 44: (894, 300)}
{43: (313, 402), 42: (94, 382), 44: (894, 300)}
{43: (313, 402), 42: (89, 381), 44: (894, 300)}
{43: (308, 402), 42: (89, 381)}
{43: (308, 402), 42: (88, 381)}
{43: (305, 402), 42: (88, 381)}
{43: (305, 402), 42: (86, 381)}
{43: (299, 402), 42: (86, 381)}
{43: (299, 402), 42: (72, 391)}
{43: (299, 402), 42: (72, 391)}
{43: (299, 402), 42: (72, 391)}
{43: (293, 401), 42: (72, 391)}
{43: (293, 401), 42: (69, 391)}
{43: (289, 400), 42: (69, 391)}
{43: (289, 400), 42: (68, 392)}
{43: (286, 401), 42: (68, 392)}
{43: (286, 401), 42: (66, 392)}
{43: (281, 399), 42: (66

{47: (357, 497), 46: (977, 491)}
{47: (357, 497), 46: (983, 490)}
{47: (357, 498), 46: (983, 490)}
{47: (357, 498), 46: (982, 490)}
{47: (349, 503), 46: (982, 490)}
{47: (406, 493), 46: (982, 490)}
{47: (406, 493), 46: (988, 488)}
{47: (363, 501), 46: (988, 488)}
{47: (363, 501), 46: (990, 487)}
{47: (373, 502), 46: (990, 487)}
{47: (373, 502), 46: (993, 485)}
{47: (392, 502), 46: (993, 485)}
{47: (392, 502), 46: (999, 484)}
{47: (384, 502), 46: (999, 484)}
{47: (384, 502), 46: (997, 484)}
{47: (393, 506), 46: (997, 484)}
{47: (393, 506), 46: (1003, 483)}
{47: (400, 503), 46: (1003, 483)}
{47: (400, 503), 46: (1005, 482)}
{47: (374, 503), 46: (1005, 482)}
{47: (374, 503), 46: (1008, 481)}
{47: (380, 507), 46: (1008, 481)}
{47: (380, 507), 46: (1012, 480)}
{47: (387, 504), 46: (1012, 480)}
{47: (387, 504), 46: (1011, 480)}
{47: (387, 509), 46: (1011, 480)}
{47: (435, 504)}
{47: (444, 504)}
{47: (425, 502)}
{47: (496, 502)}
{47: (425, 511)}
{47: (496, 503)}
{47: (505, 502)}
{47: (469, 50

{50: (869, 436)}
{50: (871, 434)}
{50: (874, 433)}
{50: (877, 430)}
{50: (876, 429)}
{50: (882, 426)}
{50: (883, 423)}
{50: (885, 421)}
{50: (889, 420)}
{50: (889, 420)}
{50: (892, 416)}
{50: (894, 414)}
{50: (895, 412)}
{50: (897, 410)}
{50: (898, 410)}
{50: (900, 407)}
{50: (902, 405)}
{50: (903, 402)}
{50: (905, 399)}
{50: (905, 399)}
{50: (907, 396)}
{50: (909, 394)}
{50: (909, 392)}
{50: (909, 390)}
{50: (909, 390)}
{50: (912, 385)}
{50: (913, 384)}
{50: (913, 383)}
{50: (914, 381)}
{50: (915, 380)}
{50: (915, 376)}
{50: (915, 374)}
{50: (915, 371)}
{50: (915, 369)}
{50: (915, 369)}
{50: (915, 366)}
{50: (916, 364)}
{50: (917, 362)}
{50: (916, 361)}
{50: (916, 360)}
{50: (916, 355)}
{50: (916, 354)}
{50: (916, 353)}
{50: (916, 350)}
{50: (916, 350)}
{50: (917, 348)}
{50: (917, 347)}
{50: (916, 344)}
{50: (915, 342)}
{50: (915, 342)}
{50: (914, 339)}
{50: (914, 337)}
{50: (912, 335)}
{50: (912, 333)}
{50: (912, 333)}
{50: (910, 330)}
{50: (910, 328)}
{50: (910, 327)}
{50: (908, 326

In [2]:
# Create tracker object
tracker2 = EuclideanDistTracker()
checker2 = CarTracker()

# create background subtractor object
backSub = cv2.createBackgroundSubtractorMOG2()

# kernel for image dilation
kernel = np.ones((4,4),np.uint8)

# Open the video file
video= cv2.VideoCapture('Traffic_Laramie_2.mp4')

# check if the video opened successfully
if (video.isOpened()== False): 
    print("Error opening video file")

# Read the first frame
ret, frame1 = video.read()

# get width and height of video frame
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# define dimensions for region of interest(ROI) rectangle
roiX = 0
roiY = int(height/2-40)
roiW = width
roiH = int(height/2+40)

# initialize car position storage
first_loc = []
next_loc = []

# Convert first frame to grayscale
prev_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# Loop through the video
while True:
    # Read the next frame
    ret, frame2 = video.read()
    
    # If there is no next frame, break out of the loop
    if not ret:
        break
    
    detections = []
    
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    # Perform frame differencing
    frame_diff = cv2.absdiff(prev_gray, gray)
    
    # apply background subtraction to ROI
    fgMask = backSub.apply(frame_diff)
    
    # remove noise
    image = cv2.GaussianBlur(fgMask, (25,25), 0)

    # apply thresholding to create binary image
    thresh = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)[1]
    
    # image dilation
    dilated = cv2.dilate(thresh,kernel,iterations = 1)
    
    # find contours in binary image
    contours, hierarchy = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # loop through contours
    for cnt in contours:
        # calculate contour area
        ctArea = cv2.contourArea(cnt)

        # draw bounding box around contour
        (x, y, w, h) = cv2.boundingRect(cnt)
        if y > height/2-40 and ctArea > 3500:
            cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 255, 0), 2)
            detections.append([x, y, w, h])

            
    cars_ids = tracker2.update(detections)
    checker2.check_x_coords(cars_ids)
    
    for car_id in cars_ids:
        x, y, w, h, id = car_id
        cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # draw ROI rectangle on frame
    cv2.rectangle(frame2, (roiX, roiY), (roiX + roiW, roiY + roiH), (0, 0, 255), 2)
    cv2.rectangle(frame2, (500, 350), (550, 450), (0, 255, 0), 2)
    
    # Display the resulting frame
    cv2.imshow('Frame Difference', dilated)
    cv2.imshow('Frame', frame2)
    
    # Wait for user input to exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the capture and destroy any OpenCV windows
video.release()
cv2.destroyAllWindows()

{0: (1014, 462)}
{0: (1013, 455)}
{0: (1011, 451)}
{0: (1010, 448)}
{0: (1008, 445)}
{0: (1007, 443)}
{0: (1005, 437)}
{0: (1004, 434)}
{0: (1002, 429)}
{0: (1001, 427)}
{0: (1001, 425)}
{0: (1000, 420)}
{0: (999, 416)}
{0: (997, 413)}
{0: (996, 410)}
{0: (995, 408)}
{0: (989, 403)}
{0: (987, 400)}
{0: (984, 396)}
{0: (982, 394)}
{0: (980, 392)}
{0: (975, 387)}
{0: (972, 384)}
{0: (970, 380)}
{0: (968, 378)}
{0: (966, 377)}
{0: (962, 373)}
{0: (960, 369)}
{0: (956, 367)}
{0: (955, 365)}
{0: (954, 364)}
{0: (950, 359)}
{0: (947, 356)}
{0: (945, 354)}
{0: (943, 352)}
{0: (941, 350)}
{0: (937, 346)}
{0: (934, 343)}
{0: (932, 341)}
{0: (931, 338)}
{0: (930, 337)}
{0: (926, 333)}
{0: (925, 331)}
{0: (924, 329)}
{0: (921, 327)}
{2: (35, 430)}
{2: (37, 432)}
{2: (39, 433)}
{2: (41, 435)}
{2: (43, 436)}
{2: (46, 437)}
{2: (48, 439)}
{2: (50, 440)}
{2: (52, 441)}
{2: (54, 443)}
{2: (57, 444)}
{2: (62, 446)}
{2: (66, 448), 3: (997, 567)}
{3: (996, 566), 2: (66, 448)}
{3: (996, 566), 2: (70, 449)

{2: (396, 530), 4: (47, 433), 3: (218, 395)}
{2: (396, 530), 4: (49, 434), 3: (218, 395)}
{2: (396, 530), 4: (49, 434), 3: (212, 389)}
{2: (397, 530), 4: (49, 434), 3: (212, 389)}
{2: (397, 530), 4: (51, 434), 3: (212, 389)}
{2: (397, 530), 4: (51, 434), 3: (206, 389)}
{2: (398, 531), 4: (51, 434), 3: (206, 389)}
{2: (398, 531), 4: (53, 435), 3: (206, 389)}
{2: (398, 531), 4: (53, 435), 3: (202, 388)}
{2: (400, 531), 4: (53, 435), 3: (202, 388)}
{2: (400, 531), 4: (56, 436), 3: (202, 388)}
{2: (400, 531), 4: (56, 436), 3: (198, 389)}
{2: (401, 531), 4: (56, 436), 3: (198, 389)}
{2: (401, 531), 4: (58, 436), 3: (198, 389)}
{2: (401, 531), 4: (58, 436), 3: (193, 386)}
{2: (402, 532), 4: (58, 436), 3: (193, 386)}
{2: (402, 532), 4: (62, 438), 3: (193, 386)}
{2: (402, 532), 4: (62, 438), 3: (186, 386)}
{2: (403, 532), 4: (62, 438), 3: (186, 386)}
{2: (403, 532), 4: (66, 440), 3: (186, 386)}
{2: (403, 532), 4: (66, 440), 3: (180, 387)}
{2: (405, 532), 4: (66, 440), 3: (180, 387)}
{2: (405, 

{8: (435, 548), 6: (1008, 503), 9: (982, 418)}
{8: (435, 548), 6: (1008, 503), 9: (976, 418)}
{8: (443, 549), 9: (976, 418)}
{8: (443, 549), 9: (972, 418)}
{8: (450, 549), 9: (972, 418)}
{8: (450, 549), 9: (968, 419)}
{8: (462, 550), 9: (968, 419)}
{8: (462, 550), 9: (964, 419)}
{8: (462, 551), 9: (964, 419)}
{8: (462, 551), 9: (960, 419)}
{8: (471, 552), 9: (960, 419)}
{8: (471, 552), 9: (952, 419)}
{8: (478, 553), 9: (952, 419)}
{8: (478, 553), 9: (947, 420)}
{8: (484, 554), 9: (947, 420)}
{8: (484, 554), 9: (943, 420)}
{8: (494, 554), 9: (943, 420)}
{8: (494, 554), 9: (940, 420)}
{8: (498, 555), 9: (940, 420)}
{8: (498, 555), 9: (933, 420)}
{8: (512, 557), 9: (933, 420)}
{8: (512, 557), 9: (926, 421)}
{8: (517, 559), 9: (926, 421)}
{8: (517, 559), 9: (921, 421)}
{8: (524, 559), 9: (921, 421)}
{8: (524, 559), 9: (915, 421)}
{8: (529, 560), 9: (915, 421)}
{8: (529, 560), 9: (911, 422)}
{8: (535, 561), 9: (911, 422)}
{8: (535, 561), 9: (905, 422)}
{8: (547, 563), 9: (905, 422)}
{8: (54

{18: (1003, 539)}
{18: (1002, 538)}
{18: (1001, 537)}
{18: (1000, 537)}
{18: (999, 537)}
{18: (999, 536)}
{18: (999, 536)}
{18: (998, 536)}
{18: (997, 535)}
{18: (997, 534)}
{18: (996, 534)}
{18: (996, 534)}
{18: (995, 533)}
{18: (995, 533)}
{18: (994, 533)}
{18: (993, 531)}
{18: (993, 531)}
{18: (992, 530)}
{18: (991, 530)}
{18: (992, 529)}
{18: (991, 528)}
{18: (991, 527)}
{18: (990, 527)}
{18: (989, 526)}
{18: (988, 527)}
{18: (988, 527)}
{18: (988, 526)}
{18: (987, 525)}
{18: (987, 523)}
{18: (986, 522)}
{18: (986, 522)}
{18: (986, 521)}
{18: (985, 520)}
{18: (984, 520)}
{18: (983, 520)}
{18: (983, 519)}
{18: (982, 519)}
{18: (981, 518)}
{18: (981, 518)}
{18: (980, 517)}
{18: (979, 517)}
{18: (979, 514)}
{18: (978, 515)}
{18: (978, 513)}
{18: (977, 515)}
{18: (976, 513)}
{18: (976, 513)}
{18: (975, 512)}
{18: (974, 512)}
{18: (974, 511)}
{18: (973, 511)}
{18: (973, 510)}
{18: (971, 510)}
{18: (971, 509)}
{18: (969, 508)}
{18: (969, 508)}
{18: (968, 508)}
{18: (967, 508)}
{18: (966,

{18: (461, 540), 20: (74, 436), 19: (964, 319)}
{18: (461, 540), 20: (77, 437), 19: (964, 319)}
{18: (461, 540), 20: (77, 437), 19: (962, 318)}
{18: (461, 541), 20: (77, 437), 19: (962, 318)}
{18: (461, 541), 20: (79, 438), 19: (962, 318)}
{18: (461, 541), 20: (79, 438), 19: (960, 316)}
{18: (463, 541), 20: (79, 438), 19: (960, 316)}
{18: (463, 541), 20: (81, 439), 19: (960, 316)}
{18: (463, 541), 20: (81, 439), 19: (958, 315)}
{18: (464, 541), 20: (81, 439), 19: (958, 315)}
{18: (464, 541), 20: (85, 440), 19: (958, 315)}
{18: (464, 541), 20: (85, 440), 19: (957, 314)}
{18: (465, 542), 20: (85, 440), 19: (957, 314)}
{18: (465, 542), 20: (87, 440), 19: (957, 314)}
{18: (465, 542), 20: (87, 440), 19: (956, 313)}
{18: (468, 542), 20: (87, 440), 19: (956, 313)}
{18: (468, 542), 20: (88, 441), 19: (956, 313)}
{18: (468, 542), 20: (88, 441), 19: (953, 312)}
{18: (471, 542), 20: (88, 441), 19: (953, 312)}
{18: (471, 542), 20: (92, 442), 19: (953, 312)}
{18: (471, 542), 20: (92, 442), 19: (951

{20: (525, 513)}
{20: (539, 514)}
{20: (550, 515)}
{20: (557, 515)}
{20: (563, 515)}
{20: (570, 515)}
{20: (586, 516)}
{20: (598, 516)}
{20: (604, 517)}
{20: (610, 518)}
{20: (617, 518)}
{20: (632, 519)}
{20: (641, 520)}
{20: (649, 520)}
{20: (657, 520)}
{20: (663, 520)}
{20: (677, 520)}
{20: (687, 520)}
{20: (695, 520)}
{20: (700, 520)}
{20: (708, 520)}
{20: (720, 520)}
{20: (733, 520)}
{20: (740, 521)}
{20: (745, 521)}
{20: (752, 520)}
{20: (766, 519)}
{20: (776, 519)}
{20: (781, 519)}
{20: (792, 518)}
{20: (797, 519)}
{20: (809, 519)}
{20: (816, 518)}
{20: (824, 517)}
{20: (829, 517)}
{20: (834, 516)}
{20: (849, 515)}
{20: (855, 514)}
{20: (861, 514)}
{20: (870, 514)}
{20: (876, 512)}
{20: (884, 513)}
{20: (890, 511)}
{20: (900, 509)}
{20: (906, 508)}
{20: (911, 506)}
{20: (920, 505)}
{20: (925, 505)}
{20: (934, 505)}
{20: (939, 504)}
{20: (945, 502)}
{20: (952, 502)}
{20: (955, 501)}
{20: (959, 501)}
{20: (962, 500)}
{20: (965, 499)}
{20: (971, 498)}
{20: (974, 497)}
{20: (978, 498

{27: (898, 377)}
{27: (900, 380)}
{27: (905, 382)}
{27: (908, 388)}
{27: (909, 391)}
{27: (913, 397)}
{27: (916, 400)}
{27: (918, 403)}
{27: (924, 409)}
{27: (926, 413)}
{27: (931, 418)}
{27: (934, 421)}
{27: (938, 425)}
{27: (942, 433)}
{27: (944, 438)}
{27: (947, 444)}
{27: (952, 448)}
{27: (957, 452)}
{27: (961, 462)}
{27: (964, 468)}
{27: (968, 474)}
{27: (973, 477)}
{27: (977, 482)}
{27: (982, 493)}
{27: (983, 500)}
{27: (986, 505)}
{27: (986, 511)}
{27: (988, 516)}
{27: (991, 522)}
{27: (993, 526)}
{27: (995, 528)}
{27: (996, 531)}
{27: (999, 533)}
{27: (1001, 539)}
{27: (1001, 543)}
{27: (1005, 546)}
{27: (1007, 550)}
{27: (1010, 550)}


## Get number of cars

In [3]:
num_cars_1 = checker1.toCityCtr
num_cars_2 = checker2.toCityCtr

In [4]:
# define a function to calculate duration of video
def dur_calc(video_file):
    vid = cv2.VideoCapture(video_file)
    
    # Get frame count
    frame_count = vid.get(cv2.CAP_PROP_FRAME_COUNT)
    
    # Get frames per second
    frame_per_sec = vid.get(cv2.CAP_PROP_FPS)
    
    # Duration of video in seconds = total frames / frames per second i.e. total frames * seconds / frames
    sec = round(frame_count / frame_per_sec)
    
    return sec

In [5]:
# define function to calculate cars going to city centre per minute
def cars_per_min(num_cars, dur_in_sec):
    # 1 minute = 60 seconds
    dur_in_min = dur_in_sec/60
    
    # cars per minute is total cars/total minutes
    cars_per_min = round(num_cars/dur_in_min)
    return cars_per_min

In [11]:
vid_dur_1 = dur_calc('Traffic_Laramie_1.mp4')
carsPerMin1 = cars_per_min(len(num_cars_1),vid_dur_1)
vid_dur_2 = dur_calc('Traffic_Laramie_2.mp4')
carsPerMin2 = cars_per_min(len(num_cars_2),vid_dur_2)

In [10]:
print("The number of cars going to city centre in Traffic_Laramie_1: {}".format(len(num_cars_1)))
print("The cars per minute going to city centre in Traffic_Laramie_1: {}".format(carsPerMin1))
print("The number of cars going to city centre in Traffic_Laramie_2: {}".format(len(num_cars_2)))
print("The cars per minute going to city centre in Traffic_Laramie_2: {}".format(carsPerMin2))

The number of cars going to city centre in Traffic_Laramie_1: 6
The cars per minute going to city centre in Traffic_Laramie_1: 2
The number of cars going to city centre in Traffic_Laramie_2: 4
The cars per minute going to city centre in Traffic_Laramie_2: 2
